In [7]:
import pandas as pd
from itertools import combinations

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.options.mode.chained_assignment = None

In [8]:
def jaccard_similarity(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    
    # Include an additional condition for a single item difference
    if abs(len(set1) - len(set2)) <= 2:
        return len(intersection) / len(union)
    else:
        return 0

def find_most_similar_lists(lists):
    max_similarity = 0
    most_similar_lists = None
    
    for pair in combinations(lists, 2):
        list1, list2 = pair
        similarity = jaccard_similarity(list1, list2)
        
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_lists = pair
    
    return most_similar_lists

In [9]:
data = pd.read_csv(fr'Run_3_FERM\All_data_metrics_main.csv')
data.drop(columns='Test_ID', inplace=True)
print(data.shape)
data.sort_values('MAE', axis=0, inplace=True)
savedInputs = data.apply(lambda row: eval(row['Inputs']), axis=1).tolist()

for n, row in enumerate(savedInputs[:-1]):
    for idx, row2 in enumerate(savedInputs[n+1:]):
        if set(row).issubset(set(row2)) or set(row2) <= set(row):
            savedInputs.remove(row2)

print("Remaining lists:")
savedInputsSTR = [repr(sublist) for sublist in savedInputs]

# Filter the DataFrame based on matching sublists
filtered_df = data[data['Inputs'].isin(savedInputsSTR)]
filtered_df.reset_index(drop=True, inplace=True)
print(filtered_df.shape)

mean_mae = filtered_df['MAE'].mean()
quart = filtered_df['MAE'].quantile(0.25)
mask = filtered_df['MAE'] >= quart
filtered_df = filtered_df[~mask]
print(filtered_df.shape)
filtered_df.to_csv(f"Run_3_FERM\Filtered_data.csv", mode = 'w', index = False)



(4016, 7)
Remaining lists:
(382, 7)
(96, 7)
